# Extract data at station points from downscaled models
## Climatologies and time series for interannual 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import re
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages

In [2]:
# data files from shared folder from daisy. 
##downloaded into large data
##deleted after points extracted

# site locations - moorings - time series
ooi_co =np.array([[47.1336, -124.272,29], # inshore: lat, lon, depth (surface always 7m)
     [46.9859, -124.566, 88], # shelf
     [46.8508, -124.972, 540]]) # offshore
locs = ['Inshore','Shelf','Offshore']

# site locations - WOA - only climatology
WOA_T = np.array([[46.875,-124.875,150],
        [47.375,-124.625,100],
        [46.375,-124.375,70]])

WOA_O = np.array([[46.5,-124.5,125],
         [47.5,-124.5,25]])

# time period to compare
itime = '2014-01-01'
ftime = '2020-12-31'

# directory to save selected data to compare
dwdir = '../data/Downscaleddata/Comparison_Obs_GCMs/'#

# directory to save figures
figdir = '../figures/Comparison_Obs_GCMs/'

# directory where downscaled data is
lpdir = '/Users/Sol/Documents/LargeData/QIN/downscaled_data/'

In [3]:
# Temperature / Oxygen
# For both scenarios
# select each IOO point

ssp = ['ssp245','ssp585']
var = ['thetao','o2']
depth = ['surface','bottom']

for ji,j in enumerate(var):
    for ki, k in enumerate(depth):
        for mi, m in enumerate(ssp):
            ddir = lpdir+m+'/'+j+'/'+k+'/'

            for i in range(3):
                # OOI coordinates
                lati = ooi_co[i,0]
                loni = ooi_co[i,1]+360

                mo = 0 # each model
                for fn in glob.glob(ddir+'*.nc'):
                    fnw1 = re.split('/',fn)
                    fnw = re.split('_',fnw1[-1])

                    # read file
                    ds = xr.open_dataset(fn)
                    ds.close()
                    # select time (years when data is available)
                    ds = ds.sel(time=slice(itime,ftime))

                    if ji == 0: # thetao
                        tmp = ds.thetao.sel(lat=slice(lati-0.1, lati+0.1), lon=slice(loni-0.1,loni+0.1))
                    else:
                        tmp = ds.o2.sel(lat=slice(lati-0.1, lati+0.1), lon=slice(loni-0.1,loni+0.1))
                        # if O2, clean any negative values
                        tmp = tmp.where(tmp>0,0)
        
                    tmp = tmp.mean(dim={'lat','lon'})

                    # save in dataframe
                    if mo == 0: # first time, create file
                        tmp2 = pd.DataFrame(data=ds.time.dt.year.values,columns={'year'})
                        tmp2['month']=ds.month.values
                        tmp2['day']=1
                        tmp2['Datetime']=pd.to_datetime(tmp2[list(tmp2)[:3]])
                        tmp2 = tmp2.set_index('Datetime')
                        tmp2.index.names=['']
                        tmp2[fnw[0]+'_'+fnw[1]] = tmp.values            
                        tmp2=tmp2.drop(['year','month','day'], axis=1)
                        tmp2.to_csv(dwdir+m+'_OOI_'+locs[i]+'_'+k+'_'+j+'.csv')
                        mo = 1
                    else: # not the first time, so open file and add column
                        tmp3 = pd.read_csv(dwdir+m+'_OOI_'+locs[i]+'_'+k+'_'+j+'.csv',index_col=0)
                        tmp3[fnw[0]+'-'+fnw[1]]=tmp.values
                        tmp3.to_csv(dwdir+m+'_OOI_'+locs[i]+'_'+k+'_'+j+'.csv')



In [4]:
# Temperature / Oxygen
# For both scenarios
# select each WOA point

#T
locs=['woa1','woa2','woa3']
ssp = ['ssp245','ssp585']
j = 'thetao'
depth = ['surface','bottom']

for ki, k in enumerate(depth):
    for mi, m in enumerate(ssp):
        ddir = lpdir+m+'/'+j+'/'+k+'/'

        for i in range(3):
            # WOA coordinates
            lati = WOA_T[i,0]
            loni = WOA_T[i,1]+360

            mo = 0 # each model
            for fn in glob.glob(ddir+'*.nc'):
                fnw1 = re.split('/',fn)
                fnw = re.split('_',fnw1[-1])

                # read file
                ds = xr.open_dataset(fn)
                ds.close()
                # select time (years when data is available)
                ds = ds.sel(time=slice(itime,ftime))

                tmp = ds.thetao.sel(lat=slice(lati-0.1, lati+0.1), lon=slice(loni-0.1,loni+0.1))
                tmp = tmp.mean(dim={'lat','lon'})

                # save in dataframe
                if mo == 0: # first time, create file
                    tmp2 = pd.DataFrame(data=ds.time.dt.year.values,columns={'year'})
                    tmp2['month']=ds.month.values
                    tmp2['day']=1
                    tmp2['Datetime']=pd.to_datetime(tmp2[list(tmp2)[:3]])
                    tmp2 = tmp2.set_index('Datetime')
                    tmp2.index.names=['']
                    tmp2[fnw[0]+'_'+fnw[1]] = tmp.values            
                    tmp2=tmp2.drop(['year','month','day'], axis=1)
                    tmp2.to_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv')
                    mo = 1
                else: # not the first time, so open file and add column
                    tmp3 = pd.read_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv',index_col=0)
                    tmp3[fnw[0]+'-'+fnw[1]]=tmp.values
                    tmp3.to_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv')

# O2
j = 'o2'
for ki, k in enumerate(depth):
    for mi, m in enumerate(ssp):
        ddir = lpdir+m+'/'+j+'/'+k+'/'

        for i in range(2):
            # WOA coordinates
            lati = WOA_O[i,0]
            loni = WOA_O[i,1]+360

            mo = 0 # each model
            for fn in glob.glob(ddir+'*.nc'):
                fnw1 = re.split('/',fn)
                fnw = re.split('_',fnw1[-1])

                # read file
                ds = xr.open_dataset(fn)
                ds.close()
                # select time (years when data is available)
                ds = ds.sel(time=slice(itime,ftime))

                tmp = ds.o2.sel(lat=slice(lati-0.1, lati+0.1), lon=slice(loni-0.1,loni+0.1))
                # if O2, clean any negative values
                tmp = tmp.where(tmp>0,0)
                tmp = tmp.mean(dim={'lat','lon'})

                # save in dataframe
                if mo == 0: # first time, create file
                    tmp2 = pd.DataFrame(data=ds.time.dt.year.values,columns={'year'})
                    tmp2['month']=ds.month.values
                    tmp2['day']=1
                    tmp2['Datetime']=pd.to_datetime(tmp2[list(tmp2)[:3]])
                    tmp2 = tmp2.set_index('Datetime')
                    tmp2.index.names=['']
                    tmp2[fnw[0]+'_'+fnw[1]] = tmp.values            
                    tmp2=tmp2.drop(['year','month','day'], axis=1)
                    tmp2.to_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv')
                    mo = 1
                else: # not the first time, so open file and add column
                    tmp3 = pd.read_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv',index_col=0)
                    tmp3[fnw[0]+'-'+fnw[1]]=tmp.values
                    tmp3.to_csv(dwdir+m+'_WOA_'+locs[i]+'_'+k+'_'+j+'.csv')

